# Install

In [1]:
# Install EfficientNet B3
!pip install '../input/kerasefficientnetb3/efficientnet-1.0.0-py3-none-any.whl'

Processing /kaggle/input/kerasefficientnetb3/efficientnet-1.0.0-py3-none-any.whl


# Importing libraries

In [2]:
import cv2
import os
import time, gc
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model, Input
from keras.layers import Dense, Lambda
from math import ceil
import matplotlib.pyplot as plt
import efficientnet.keras as efn

Using TensorFlow backend.


# Constants

In [3]:
# Ширина изображения
IMAGE_WIDTH = 236
# Высота изображения
IMAGE_HEIGHT = 137
# Коэффициент масштабирования
FACTOR = 0.7
# Каналы
CHANNELS = 3
# Размер пакета
BATCH_SIZE = 16
# Новая ширина изображения
IMAGE_WIDTH_NEW = int(IMAGE_WIDTH * FACTOR)
# Новая высота изображения
IMAGE_HEIGHT_NEW = int(IMAGE_HEIGHT * FACTOR)

# Image processing

In [4]:
def imageProcessing(image):
    """
    Обработка изображения.
    
    image - изображение
    return - обработанное изображение
    """
    
    # Invert
    image = 255 - image

    # Normalize
    image = (image * (255.0 / image.max())).astype(np.uint8)

    # Resize
    image = image.reshape(IMAGE_HEIGHT, IMAGE_WIDTH)
    image = cv2.resize(image, (IMAGE_WIDTH_NEW, IMAGE_HEIGHT_NEW), interpolation = cv2.INTER_AREA)

    return image  

# Create Model

In [5]:
# Generalized mean pool - GeM
gmExp = tf.Variable(3.0, dtype = tf.float32)
def generalizedMeanPool(X):
    pool = (tf.reduce_mean(tf.abs(X**(gmExp)), axis = [1, 2], keepdims = False) + 1.e-7)**(1./gmExp)
    
    return pool

In [6]:
def createModel(inputShape):
    """
    Создание модели блока свертки.
    
    inputShape - размер и канальность изображения
    return - модель
    """
    
    # Входной слой
    input = Input(shape=inputShape)
    # Создание и компиляция модели
    xModel = efn.EfficientNetB3(weights=None, include_top=False, input_tensor=input, pooling=None, classes=None)
    
    # Разморозка всех слоев
    for layer in xModel.layers:
        layer.trainable = True
    
    # Обобщенный средний пул
    lambdaLayer = Lambda(generalizedMeanPool)
    lambdaLayer.trainable_weights.extend([gmExp])
    dense = lambdaLayer(xModel.output)
    
    # Выход нейронной сети отвечающий за классификацию графем
    graphemeRoot = Dense(168, activation = 'softmax', name = 'root')(dense)
    # Выход нейронной сети отвечающий за классификацию гласных диакректических знаков
    vowelDiacritic = Dense(11, activation = 'softmax', name = 'vowel')(dense)
    # Выход нейронной сети отвечающий за классификацию согласных диакректических знаков
    consonantDiacritic = Dense(7, activation = 'softmax', name = 'consonant')(dense)

    # Создание модели
    model = Model(inputs = xModel.input, outputs = [graphemeRoot, vowelDiacritic, consonantDiacritic])

    return model

In [7]:
# Создание моделей
model1 = createModel(inputShape = (IMAGE_HEIGHT_NEW, IMAGE_WIDTH_NEW, CHANNELS))
model2 = createModel(inputShape = (IMAGE_HEIGHT_NEW, IMAGE_WIDTH_NEW, CHANNELS))
model3 = createModel(inputShape = (IMAGE_HEIGHT_NEW, IMAGE_WIDTH_NEW, CHANNELS))
model4 = createModel(inputShape = (IMAGE_HEIGHT_NEW, IMAGE_WIDTH_NEW, CHANNELS))
model5 = createModel(inputShape = (IMAGE_HEIGHT_NEW, IMAGE_WIDTH_NEW, CHANNELS))

In [8]:
# Загрузка весовых коэффициентов моделей
model1.load_weights("../input/kerasefficientnetb3/Train1_model_59.h5")
model2.load_weights("../input/kerasefficientnetb3/Train1_model_66.h5")
model3.load_weights("../input/kerasefficientnetb3/Train1_model_68.h5")
model4.load_weights("../input/kerasefficientnetb3/Train1_model_57.h5")
model5.load_weights("../input/kerasefficientnetb3/Train1_model_70.h5")

# Data Generator

In [9]:
class TestDataGenerator(keras.utils.Sequence):
    def __init__(self, X, batch_size = 16, img_size = (512, 512, 3), *args, **kwargs):
        self.X = X
        self.indices = np.arange(len(self.X))
        self.batch_size = batch_size
        self.img_size = img_size
                    
    def __len__(self):
        return int(ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indices)
        return X
    
    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        
        for i, index in enumerate(indices):
            image = self.X[index]
            image = np.stack((image,)*CHANNELS, axis=-1)
            image = image.reshape(-1, IMAGE_HEIGHT_NEW, IMAGE_WIDTH_NEW, CHANNELS)
            
            X[i,] = image
        
        return X

# Predict and Submission

In [10]:
components = ['grapheme_root','vowel_diacritic','consonant_diacritic']

# Список предсказаний модели
targets = []
# Список id меток предсказаний
rowIds = []
# Количество файлов
numberFiles = 4

# Цикл по тестовым файлам
for i in range(numberFiles):
    # Test Files Placeholder
    testFiles = []
    
    # Загрузка тестового набора изображений
    df = pd.read_parquet(f"/kaggle/input/bengaliai-cv19/test_image_data_{i}.parquet")
    # Получение идентификаторов изображений
    imageIds = df['image_id'].values 
    # Удаление ключа 'image_id'
    df = df.drop(['image_id'], axis = 1)
    
    X = []
    # Цикл обработки строк фрейма данных
    for imageId, index in zip(imageIds, range(df.shape[0])):
        # Добавление идентификатора изображения в список
        testFiles.append(imageId)
        # Получение изображения
        image = df.loc[df.index[index]].values
        # Обработка изображения
        image = imageProcessing(image)
        # Добавление преобразованного изображения в список
        X.append(image.reshape(-1))
    
    # Генератор данных
    dataGeneratorTest = TestDataGenerator(X, batch_size=BATCH_SIZE, img_size=(IMAGE_HEIGHT_NEW, IMAGE_WIDTH_NEW, CHANNELS))
        
    # Выполнение предсказания
    preds1 = model1.predict_generator(dataGeneratorTest, verbose = 1)
    preds2 = model2.predict_generator(dataGeneratorTest, verbose = 1)
    preds3 = model3.predict_generator(dataGeneratorTest, verbose = 1)
    preds4 = model4.predict_generator(dataGeneratorTest, verbose = 1)
    preds5 = model5.predict_generator(dataGeneratorTest, verbose = 1)
    
    # Цикл после прогнозирования    
    for i, imageId in zip(range(len(testFiles)), testFiles):
        for subi, col in zip(range(len(preds1)), components):
            subPreds1 = preds1[subi]
            subPreds2 = preds2[subi]
            subPreds3 = preds3[subi]
            subPreds4 = preds4[subi]
            subPreds5 = preds5[subi]
            
            rowIds.append(str(imageId) + '_' + col)
            # Установка прозноза со средним значением пяти прогнозов
            subPredValue = np.argmax((subPreds1[i] + subPreds2[i] + subPreds3[i] + subPreds4[i] + subPreds5[i]) / 5)
            targets.append(subPredValue)
    
    # Очистка
    del df
    gc.collect()

1/1 [==============================] - 0s 44ms/step


In [11]:
# Создание и сохранение файла ответов
submit_df = pd.DataFrame({'row_id':rowIds, 'target':targets}, columns = ['row_id', 'target'])
submit_df.to_csv('submission.csv', index = False)
print(submit_df.head(40))

                         row_id  target
0          Test_0_grapheme_root       3
1        Test_0_vowel_diacritic       0
2    Test_0_consonant_diacritic       0
3          Test_1_grapheme_root      93
4        Test_1_vowel_diacritic       2
5    Test_1_consonant_diacritic       0
6          Test_2_grapheme_root      19
7        Test_2_vowel_diacritic       0
8    Test_2_consonant_diacritic       0
9          Test_3_grapheme_root     115
10       Test_3_vowel_diacritic       0
11   Test_3_consonant_diacritic       0
12         Test_4_grapheme_root      55
13       Test_4_vowel_diacritic       4
14   Test_4_consonant_diacritic       0
15         Test_5_grapheme_root     115
16       Test_5_vowel_diacritic       2
17   Test_5_consonant_diacritic       0
18         Test_6_grapheme_root     147
19       Test_6_vowel_diacritic       9
20   Test_6_consonant_diacritic       5
21         Test_7_grapheme_root     137
22       Test_7_vowel_diacritic       7
23   Test_7_consonant_diacritic       0


In [12]:
# Сохранение моделей
model1.save("model1.h5")
model2.save("model2.h5")
model3.save("model3.h5")
model4.save("model4.h5")
model5.save("model5.h5")